In [ ]:
from PIL import Image
import os
from collections import Counter
import time

def get_image_info(directory):
    image_info = []
    width_distribution = Counter()
    height_distribution = Counter()
    resolution_distribution = Counter()

    max_resolution = 0
    min_resolution = float('inf')
    max_res_file = ""
    min_res_file = ""

    max_width = 0
    min_width = float('inf')
    max_width_file = ""
    min_width_file = ""

    max_height = 0
    min_height = float('inf')
    max_height_file = ""
    min_height_file = ""

    for filename in os.listdir(directory):
        if filename.endswith('.jpg') or filename.endswith('.png'):
            path = os.path.join(directory, filename)
            with Image.open(path) as img:
                width, height = img.size
                resolution = img.width * img.height

                # 更新宽度统计
                if width > max_width:
                    max_width = width
                    max_width_file = filename
                if width < min_width:
                    min_width = width
                    min_width_file = filename

                # 更新高度统计
                if height > max_height:
                    max_height = height
                    max_height_file = filename
                if height < min_height:
                    min_height = height
                    min_height_file = filename

                # 更新分辨率统计
                if resolution > max_resolution:
                    max_resolution = resolution
                    max_res_file = filename
                
                if resolution < min_resolution:
                    min_resolution = resolution
                    min_res_file = filename

                # 分布统计
                width_distribution[width // 100 * 100] += 1
                height_distribution[height // 100 * 100] += 1
                resolution_distribution[resolution // 50000 * 50000] += 1

                # 收集图片信息
                info = {
                    'filename': filename,
                    'width': width,
                    'height': height,
                    'resolution': resolution
                }
                image_info.append(info)

    # 打印统计数据
    print("Width Distribution (by 100 pixels):")
    for w, count in sorted(width_distribution.items()):
        print(f"{w} to {w+99} pixels: {count} images")
    print(f"Max Width: {max_width} pixels (File: {max_width_file}), Min Width: {min_width} pixels (File: {min_width_file})")

    print("Height Distribution (by 100 pixels):")
    for h, count in sorted(height_distribution.items()):
        print(f"{h} to {h+99} pixels: {count} images")
    print(f"Max Height: {max_height} pixels (File: {max_height_file}), Min Height: {min_height} pixels (File: {min_height_file})")

    print("Resolution Distribution (by 1 MP):")
    for r, count in sorted(resolution_distribution.items()):
        print(f"{r} to {r+49999} pixels: {count} images")
    print(f"Max Resolution: {max_resolution} pixels (File: {max_res_file}), Min Resolution: {min_resolution} pixels (File: {min_res_file})")
    
    return image_info




# 使用示例
directory = '../data/source/trainning set/png'
image_info = get_image_info(directory)
image_info

In [ ]:
from PIL import Image, ImageOps
import os

def pad_images(directory, image_info, save_directory):
    idx = 0
    for info in image_info:
        path = os.path.join(directory, info['filename'])
        idx += 1
        if idx % 10 == 0:
            print(f"Padding images: {idx}/{len(image_info)} finished")
        with Image.open(path) as img:
            padding_width = int(img.width * 0.05)
            padding_height = int(img.height * 0.05)
            padded_img = ImageOps.expand(img, border=(padding_width, padding_height), fill='white')
            # 保存填充后的图片
            save_path = os.path.join(save_directory, f"padded_{info['filename']}")
            padded_img.save(save_path)
            
            
def resize_images(directory, target_height=256, save_directory='../data/resized'):
    file_list = [f for f in os.listdir(directory) if f.startswith('padded_')]
    idx = 0
    for filename in file_list:
        idx += 1
        if idx % 10 == 0:
            print(f"Resizing images: {idx}/{len(file_list)} finished")
        path = os.path.join(directory, filename)
        with Image.open(path) as img:
            aspect_ratio = img.width / img.height
            new_width = int(target_height * aspect_ratio)
            resized_img = img.resize((new_width, target_height), Image.ANTIALIAS)
            save_path = os.path.join(save_directory, f"{filename[len('padded_'):]}")
            resized_img.save(save_path)


In [ ]:
import os
import cv2
import numpy as np

def add_gaussian_noise_and_save_batch(input_directory, output_directory, mean=0, var=0.01):
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)
    
    image_files = [f for f in os.listdir(input_directory) if os.path.isfile(os.path.join(input_directory, f))]
    total_images = len(image_files)
    
    idx = 0
    for image_file in image_files:
        path = os.path.join(input_directory, image_file)
        idx += 1
        if idx % 10 == 0 or idx == total_images:
            print(f"Processing images: {idx}/{total_images} finished")
        
        image = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        
        # 确保图像高度为128
        if image.shape[0] != 128:
            raise ValueError(f"Image {image_file} height must be 128 pixels")
        
        # 计算标准差
        sigma = var ** 0.5
        
        # 生成高斯噪声
        gauss = np.random.normal(mean, sigma, image.shape)
        
        # 添加噪声到图像
        noisy_image = image + gauss
        
        # 裁剪像素值到有效范围
        noisy_image = np.clip(noisy_image, 0, 255)
        
        # 转换为无符号8位整数类型
        noisy_image = noisy_image.astype(np.uint8)
        
        # 保存图像
        save_path = os.path.join(output_directory, f"{image_file}")
        cv2.imwrite(save_path, noisy_image)

# 示例使用
# add_gaussian_noise_and_save_batch('input_folder', 'output_folder')

In [ ]:
import os
import cv2
import numpy as np

def sharpen_image_and_save_batch(input_directory, output_directory):
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)
    
    image_files = [f for f in os.listdir(input_directory) if os.path.isfile(os.path.join(input_directory, f))]
    total_images = len(image_files)
    
    idx = 0
    for image_file in image_files:
        path = os.path.join(input_directory, image_file)
        idx += 1
        if idx % 10 == 0 or idx == total_images:
            print(f"Processing images: {idx}/{total_images} finished")
        
        image = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        
        # 确保图像高度为128
        if image.shape[0] != 128:
            raise ValueError(f"Image {image_file} height must be 128 pixels")
        
        # 创建锐化内核
        kernel = np.array([[0, -1, 0], 
                           [-1, 5, -1], 
                           [0, -1, 0]])
        
        # 应用锐化内核
        sharpened_image = cv2.filter2D(image, -1, kernel)
        
        # 保存图像
        save_path = os.path.join(output_directory, f"{image_file}")
        cv2.imwrite(save_path, sharpened_image)

# 示例使用
# sharpen_image_and_save_batch('input_folder', 'output_folder')

In [10]:
def main():
    start_time = time.time()
    source_directory = '../data/source/trainning set/png'
    padded_directory = '../data/padding'
    resized_directory = '../data/resized'
    # gaussian_noise_directory = '../data/gauss'
    sharpen_directory = '../data/sharpen'
    # sharpen_gauss_disrectory = '../data/gauss_sharpen'
    
    # 确保保存目录存在
    os.makedirs(padded_directory, exist_ok=True)
    os.makedirs(resized_directory, exist_ok=True)
    os.makedirs(sharpen_directory, exist_ok=True)
    # os.makedirs(gaussian_noise_directory, exist_ok=True)
    
    # 获取图片信息
    image_info = get_image_info(source_directory)
    print(f"Image information collection completed. Time elapsed: {time.time() - start_time} seconds")
    
    # 应用padding并保存
    pad_images(source_directory, image_info, padded_directory)
    
    print(f"Padding completed. Time elapsed: {time.time() - start_time} seconds")
    
    # 从保存的padding图片读取，调整大小并保存
    resize_images(padded_directory, 128, resized_directory)
    
    print(f"Resizing completed. Total time elapsed: {time.time() - start_time} seconds")
    
    # add_gaussian_noise_and_save_batch(resized_directory, gaussian_noise_directory)
    # add_gaussian_noise_and_save_batch(sharpen_directory, sharpen_gauss_disrectory)
    
    print(f"Gaussian noise addition completed. Total time elapsed: {time.time() - start_time} seconds")
    
    sharpen_image_and_save_batch(resized_directory, sharpen_directory)
    
    print("All processes are completed.")

main()


Padding images: 60/15000 finished
Padding images: 70/15000 finished
Padding images: 80/15000 finished
Padding images: 90/15000 finished
Padding images: 100/15000 finished
Padding images: 110/15000 finished
Padding images: 120/15000 finished
Padding images: 130/15000 finished
Padding images: 140/15000 finished
Padding images: 150/15000 finished
Padding images: 160/15000 finished
Padding images: 170/15000 finished
Padding images: 180/15000 finished
Padding images: 190/15000 finished
Padding images: 200/15000 finished
Padding images: 210/15000 finished
Padding images: 220/15000 finished
Padding images: 230/15000 finished
Padding images: 240/15000 finished
Padding images: 250/15000 finished
Padding images: 260/15000 finished
Padding images: 270/15000 finished
Padding images: 280/15000 finished
Padding images: 290/15000 finished
Padding images: 300/15000 finished
Padding images: 310/15000 finished
Padding images: 320/15000 finished
Padding images: 330/15000 finished
Padding images: 340/1500

In [11]:
directory = '../data/resized'
image_info = get_image_info(directory)
image_info

Width Distribution (by 100 pixels):
0 to 99 pixels: 33 images
100 to 199 pixels: 614 images
200 to 299 pixels: 1969 images
300 to 399 pixels: 2468 images
400 to 499 pixels: 2462 images
500 to 599 pixels: 2021 images
600 to 699 pixels: 1602 images
700 to 799 pixels: 1183 images
800 to 899 pixels: 819 images
900 to 999 pixels: 593 images
1000 to 1099 pixels: 418 images
1100 to 1199 pixels: 268 images
1200 to 1299 pixels: 182 images
1300 to 1399 pixels: 127 images
1400 to 1499 pixels: 101 images
1500 to 1599 pixels: 56 images
1600 to 1699 pixels: 42 images
1700 to 1799 pixels: 16 images
1800 to 1899 pixels: 10 images
1900 to 1999 pixels: 8 images
2000 to 2099 pixels: 3 images
2100 to 2199 pixels: 2 images
2200 to 2299 pixels: 2 images
2400 to 2499 pixels: 1 images
Max Width: 2436 pixels (File: 0051656.png), Min Width: 53 pixels (File: 0115819.png)
Height Distribution (by 100 pixels):
100 to 199 pixels: 15000 images
Max Height: 128 pixels (File: 0056466.png), Min Height: 128 pixels (File: 

[{'filename': '0056466.png',
  'width': 785,
  'height': 128,
  'resolution': 100480},
 {'filename': '0075838.png',
  'width': 839,
  'height': 128,
  'resolution': 107392},
 {'filename': '0101114.png', 'width': 225, 'height': 128, 'resolution': 28800},
 {'filename': '0102703.png',
  'width': 802,
  'height': 128,
  'resolution': 102656},
 {'filename': '0020337.png',
  'width': 981,
  'height': 128,
  'resolution': 125568},
 {'filename': '0079013.png', 'width': 281, 'height': 128, 'resolution': 35968},
 {'filename': '0004652.png', 'width': 721, 'height': 128, 'resolution': 92288},
 {'filename': '0098587.png', 'width': 721, 'height': 128, 'resolution': 92288},
 {'filename': '0021625.png', 'width': 271, 'height': 128, 'resolution': 34688},
 {'filename': '0072270.png', 'width': 265, 'height': 128, 'resolution': 33920},
 {'filename': '0125384.png', 'width': 300, 'height': 128, 'resolution': 38400},
 {'filename': '0033048.png', 'width': 537, 'height': 128, 'resolution': 68736},
 {'filename'